In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('../data/price.csv', index_col=0, parse_dates=True)
log_returns = np.log(df).diff()
log_returns.dropna(inplace=True)

# Copula Estimation
We'll fit our log returns of SPY and TLT using a Gaussian copula. As noted in notebook 1, a Student's t-distribution fit both marginals better, so we'll use t-distributions again when fitting the marginals for our copula.

In [2]:
from copulas.univariate.student_t import StudentTUnivariate
from copulas.multivariate import GaussianMultivariate

copula = GaussianMultivariate(distribution=StudentTUnivariate)
copula.fit(log_returns)

In [3]:
# Generate our data for the copula
synthetic_data = copula.sample(len(log_returns));

In [4]:
# Comparison between the simulated and empirical data
from copulas.visualization import compare_2d
compare_2d(log_returns, synthetic_data)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Data=Real<br>SPY=%{x}<br>TLT=%{y}<extra></extra>',
              'legendgroup': 'Real',
              'marker': {'color': '#000036', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Real',
              'showlegend': True,
              'type': 'scattergl',
              'x': {'bdata': ('AOuwgY6pkr8AuhCpvmGDvwD6oroTXY' ... '3kEsR3vwCU6joCI3A/AKCjJepVUr8='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('AICz2Mjrjz8AMG5eCEmSPwAQyfvHMW' ... 'WubrdxvwAaS2rv3YI/AMA4M1u0WD8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'Data=Synthetic<br>SPY=%{x}<br>TLT=%{y}<extra></extra>',
              'legendgroup': 'Synthetic',
              'marker': {'color': '#01E0C9', 'symbol': 'diamond'},
              'mode': 'markers',
              'name': 'Synthetic',
              'showlegend': True,
              'type': 'scattergl',
              'x': {'bdata': ('Sas69LwMgL/2I/11lapsvzZlQ6rf13' ... '58USWQP5O5PLoz51a/YEcBX4Weir8='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('2Es4XaxaIL/k64nuXCZwPxpWoB7XGG' ... 'mANTWTP2dH95BTuog/7gIBx60XYr8='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'font': {'size': 18},
               'legend': {'title': {'text': 'Data'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'plot_bgcolor': '#F5F5F8',
               'showlegend': True,
               'template': '...',
               'title': {'text': "Real vs. Synthetic Data for columns 'SPY' and 'TLT'"},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'SPY'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'TLT'}}}
})

In [5]:
copula_params = copula.to_dict()
copula_params.keys()
copula_params['univariates']

[{'df': np.float64(2.6451729590405284),
  'loc': np.float64(0.0008998832954342148),
  'scale': np.float64(0.0064670435817185245),
  'type': 'copulas.univariate.student_t.StudentTUnivariate'},
 {'df': np.float64(6.372633378174145),
  'loc': np.float64(2.646963332306865e-05),
  'scale': np.float64(0.007873110709190386),
  'type': 'copulas.univariate.student_t.StudentTUnivariate'}]

In [6]:
copula_params['correlation']

[[1.0, -0.16294625646172714], [-0.16294625646172714, 1.0]]

# Risk Analysis
We will perform risk analysis on simulated and empirical portfolios with 50% in each asset

In [7]:
def calculate_var_es(log_returns: pd.DataFrame, weights: np.ndarray, confidence_level: float = 0.99):
    """
    Calculate Value-at-Risk (VaR) and Expected Shortfall (ES) for a two-asset portfolio.

    Parameters:
    - log_returns: pd.DataFrame with shape (n_days, n_assets), log returns of assets (e.g., ['SPY', 'TLT'])
    - weights: np.ndarray of shape (n_assets,), portfolio weights (should sum to 1)
    - confidence_level: float, confidence level for VaR/ES (e.g., 0.99 for 99%)

    Returns:
    - var: float, Value-at-Risk at the given confidence level
    - es: float, Expected Shortfall (Conditional VaR) at the given confidence level
    """
    # Compute portfolio returns
    portfolio_returns = np.exp(log_returns) @ weights
    sorted_returns = np.sort(portfolio_returns)
    # Compute VaR and ES
    var_index = int((1 - confidence_level) * len(sorted_returns))
    var = sorted_returns[var_index]
    es = sorted_returns[:var_index].mean()

    return var, es

In [8]:
weights = [0.5, 0.5]

var_empirical, es_empirical = calculate_var_es(log_returns, weights=weights)
var_simulated, es_simulated = calculate_var_es(synthetic_data, weights=weights)

print(f'Empirical: VaR: {var_empirical}, ES: {es_empirical}')
print(f'Simulated: VaR: {var_simulated}, ES: {es_simulated}')

Empirical: VaR: 0.9821356689379099, ES: 0.9746270683830379
Simulated: VaR: 0.9845766821552686, ES: 0.9721333257011546
